# 自分で実装
ソルバを使わずに自力で実装．茨の道．  
* 参考文献
<a name="1">小郷寛，美多勉，"システム制御理論入門"，実教出版株式会社，1979年</a>

In [46]:
using Plots
using LinearAlgebra

リカッチ代数方程式
$$
PA+A^{\top}P-PBR^{-1}B^{\top}P+Q=0
$$
の正定解$P$を求める．  

In [47]:
# テスト用のシステムを定義
test = (
    A = [
        1.1 2
        -0.3 -1
    ],
    B = [
        1 2
        0.847 3
    ],
    C = [
        1. 0.
        0. 1.
    ],
    Q = diagm([10.0, 10.0]),
    R = diagm([1., 1.])
);

ソルバを使って求めたリカッティ方程式の解

In [48]:
using MatrixEquations
P_solver, _, _ = arec(test.A, test.B, test.R, test.Q, zero(test.B))
P_solver

2×2 Matrix{Float64}:
  3.34568  -1.07266
 -1.07266   1.30235

# リカッチの非線形行列微分方程式を解いて求める
[教科書](#1)のp.160にある方法．力技．こちらは簡単．  

In [49]:
"""離散時間リカッティ方程式を解く"""
function solve_dare(A, B, Q, R, max=10000000, step=0.01)
    P = zero(Q)  # 初期値
    P_old = zero(Q)

    for i in 1:max
        copy!(P_old, P)
        P = P .+ (P*A .+ A'*P .- P*B*inv(R)*B'*P .+ Q) .* step
        if abs.(P_old .- P) |> maximum < 1e-6
            break
        end
    end

    P
end

P = solve_dare(test.A, test.B, test.Q, test.R)
P

2×2 Matrix{Float64}:
  3.34567  -1.07265
 -1.07265   1.30235

ソルバを使って求めた解と比較

In [50]:
P - P_solver

2×2 Matrix{Float64}:
 -1.64684e-5   7.15116e-6
  7.15116e-6  -3.10528e-6

ほぼ合ってる．  

****
## 有本・ポッターの方法
[教科書](#1)のp.160にある方法．  
まずはハミルトン行列$H$をつくる．  
$$
H = \begin{bmatrix}
A & -BR^{-1}B^{\top}\\
-Q & -A^{\top}
\end{bmatrix}
$$

In [51]:
A, B, C, Q, R = test
ℋ = [
    A -B*inv(R)*B'
    -Q -A'
]  #  ハミルトン行列
ℋ

4×4 Matrix{Float64}:
   1.1    2.0  -5.0    -6.847
  -0.3   -1.0  -6.847  -9.71741
 -10.0   -0.0  -1.1     0.3
  -0.0  -10.0  -2.0     1.0

ハミルトン行列の固有値，固有ベクトルを求める．  

In [52]:
λ = eigvals(ℋ)  # ハミルトン行列の固有値
ω = eigvecs(ℋ)  # ハミルトン行列の固有ベクトル

scatter(real(λ), imag(λ), xlabel="Re", ylabel="Im", label="eigvals")
savefig("ReIm.png")

![1](ReIm.png)

In [53]:
λ  # 固有値

4-element Vector{Float64}:
 -11.862446758953242
  -2.732479989130699
   2.732479989130694
  11.862446758953245

In [63]:
ω  # 固有ベクトル

4×4 Matrix{Float64}:
 0.304554  -0.100982   0.318186   -0.574726
 0.701358   0.360764   0.0677325  -0.540943
 0.266623  -0.724832  -0.789496    0.452973
 0.586733   0.578162   0.520448    0.414592

最適極は左半面にあるもの全てである．
今回は-11.86...と-2.73...の２つが最適極となる．  
$i$番目の最適極の固有ベクトル$\omega_i$を2つに分割する．  
$$
\omega_i = \begin{bmatrix}
u_i\\
v_i
\end{bmatrix}
$$

In [54]:
# -11.16...
u1 = ω[1:2, 1]
v1 = ω[3:4, 1]

# -2.73...
u2 = ω[1:2, 2]
v2 = ω[3:4, 2];

リカッチ解は次式となる．  
$$
P = [v_1, v_2, ...,v_n][u_1, u_2, ...,u_n]^{-1}
$$

In [55]:
P_ap = [v1 v2] / [u1 u2]
P_ap

2×2 Matrix{Float64}:
  3.34568  -1.07266
 -1.07266   1.30235

他の方法で求めたものと比較する．  

In [56]:
P_solver - P_ap

2×2 Matrix{Float64}:
 -4.44089e-16  1.11022e-15
 -1.33227e-15  0.0

以上を一つの関数にまとめたもの⇓

In [57]:
"""有本・ポッター"""
function arimoto_potter(A, B, Q, R)
    
    n = size(A)[1]
    
    ℋ = [
        A -B*inv(R)*B'
        -Q -A'
    ]  #  ハミルトン行列

    λ = eigvals(ℋ)  # ハミルトン行列の固有値
    ω = eigvecs(ℋ)  # ハミルトン行列の固有ベクトル

    U = Matrix{Complex{Float64}}(undef, n, n)
    V = Matrix{Complex{Float64}}(undef, n, n)

    i = 1
    for (l, w) in zip(λ, ω)
        if real(l) < 0
            U[:, i] = ω[1:n, i]
            V[:, i] = ω[n+1:end, i]
            i += 1
        end
    end

    (V * inv(U)) |> real
end

P_arimoto = arimoto_potter(test.A, test.B, test.Q, test.R)
P_arimoto

2×2 Matrix{Float64}:
  3.34568  -1.07266
 -1.07266   1.30235

****
## 速度比較
微分方程式を解く方法のと有本・ポッター法を比較してみる．  

In [58]:
using BenchmarkTools

function bench(N)
    A = rand(N, N)
    B = rand(N, N)
    Q = diagm(rand(N))
    R = diagm(rand(N))
    A, B, Q, R
end


bench (generic function with 1 method)

N = 3

In [59]:
A, B, Q, R = bench(3)
@benchmark solve_dare(A, B, Q, R)

BenchmarkTools.Trial: 619 samples with 1 evaluation.
 Range (min … max):  7.337 ms … 19.068 ms  ┊ GC (min … max): 0.00% … 27.51%
 Time  (median):     7.887 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   8.057 ms ±  1.034 ms  ┊ GC (mean ± σ):  0.54% ±  2.89%

   ▅▅▃█▆▆█                                                    
  █████████▇▆▄▅▅▄▃▃▃▃▃▁▁▂▂▂▁▁▁▁▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▂ ▃
  7.34 ms        Histogram: frequency by time        12.8 ms <

 Memory estimate: 818.75 KiB, allocs estimate: 5240.

In [60]:
@benchmark arimoto_potter(A, B, Q, R)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  50.300 μs …  5.234 ms  ┊ GC (min … max): 0.00% … 97.30%
 Time  (median):     56.900 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   61.730 μs ± 54.609 μs  ┊ GC (mean ± σ):  0.83% ±  0.97%

  ▇█▇██▇▆▅▄▄▄▃▂▂▁▁    ▁▁▁  ▁                                  ▃
  █████████████████▇████████████▆▇▆█▆█▅▇▆▅▇▅▅▃▃▅▄▅▁▅▅▅▆▇▇▆▇▆▆ █
  50.3 μs      Histogram: log(frequency) by time       147 μs <

 Memory estimate: 15.88 KiB, allocs estimate: 96.

有本・ポッターの方が高速．  
N = 50

In [61]:
A, B, Q, R = bench(50)
@benchmark solve_dare(A, B, Q, R)

BenchmarkTools.Trial: 59 samples with 1 evaluation.
 Range (min … max):  65.398 ms … 107.756 ms  ┊ GC (min … max):  0.00% … 16.91%
 Time  (median):     84.864 ms               ┊ GC (median):    14.37%
 Time  (mean ± σ):   85.698 ms ±  10.280 ms  ┊ GC (mean ± σ):  11.74% ±  5.90%

                             █▂       ▅                         
  █▁▁▅▁▅▅▁▁█▁▁▅▁█▅█▁██▅▁▁▅▁▁▅███▅█▅▅▁▅█▅▅▅▅▁▅▅▅█▁▅▁▅▅█▁▅▁█▁▁▁▅ ▁
  65.4 ms         Histogram: frequency by time          105 ms <

 Memory estimate: 80.65 MiB, allocs estimate: 8410.

In [62]:
@benchmark arimoto_potter(A, B, Q, R)

BenchmarkTools.Trial: 272 samples with 1 evaluation.
 Range (min … max):  12.725 ms … 28.637 ms  ┊ GC (min … max): 0.00% … 25.66%
 Time  (median):     18.552 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   18.392 ms ±  1.809 ms  ┊ GC (mean ± σ):  0.27% ±  2.05%

                          ▂ ▄   █▁▆█▂▆█▆▄                      
  ▃▁▁▁▁▄▁▃▁▃▄▁▁▄▄▅▄▄▄▇▅▆▇██▇█▅███████████▇▇▅▅▅▄▃▁▃▄▃▃▁▁▁▁▃▁▁▃ ▄
  12.7 ms         Histogram: frequency by time        23.4 ms <

 Memory estimate: 1.04 MiB, allocs estimate: 874.